# Masking the niftis 

This notebook loads in the nifti's created in the previous notebook (1_export_ciftis). Here, we'll apply different maskings to these niftis and save these as separate files. This way, the diferent maskings can be easily viewed (i.e. in fslview).

In [197]:
from __future__ import division
import os
import nibabel as nb
import numpy as np

### define directories


In [198]:
repo_dir = '/home/vanes/git/hcp_cerebellum/'
data_dir = '/home/shared/2018/visual/hcp_cerebellum/'

### subdirectories

In [199]:
mask_dir = os.path.join(data_dir,'masked_niftis')
if not os.path.isdir(mask_dir): os.mkdir(mask_dir)
resource_dir = os.path.join(os.getcwd(),'resources')

### define dimensions

In [188]:
dims = {
    'ang':0,
    'ecc':1,
    'gain':2,
    'meanvol':3,
    'r2':4,
    'rfsize':5
}

### setup functions to create different masks

In [189]:
def create_r2_mask(sj,data):
    """
    These values were determined in the original HCP retinotopy paper
    by fitting a gaussian mixture model to the distribution of R2s 
    and finding the point where two gaussians intersect (i.e. maximally
    separating a 'noise' from a 'signal' distribution)
    """
    
    if sj in [181,182,183]: # there are the different average subjects
        r2thresh=9.8         
    else: # this is for individual subjects
        r2thresh=2.2

    r2mask = (data[:,:,:,dims['r2']]>r2thresh)

    return r2mask

def create_spill_mask():
    """
    This applies the spill mask drawn on data from the avg subject.
    The 'spill' refers to activity from cortex that is averaged
    into the cerebellum by the smoothing and nonlinear transformations
    to MNI space. The mask is available in the repo of this code.
    """   
    # load the spillover mask results nifti
    fn = os.path.join(resource_dir,'volume_masks',spillovermask.nii.gz')
    img = nb.load(fn)
    spillmask = (img.get_data()==0) # spillovermask is 0 for valid voxels

    return spillmask

def create_fix_mask(sj,data):
    """
    Create a fixation mark mask, where voxels smaller than < 0.15 ecc and size
    are excluded.
    """

    sizethresh = 0.15
    eccthresh = 0.15
    sizemask = (data[:,:,:,dims['rfsize']]<sizethresh) # invalid voxels
    eccmask = (data[:,:,:,dims['ecc']]<eccthresh) # invalid voxels
    fixmask = np.invert(sizemask*eccmask) # valid voxels

    return fixmask

def create_roi_mask():
    """
    This mask deselects all voxels outside the retinotopic
    clusters identified in the average subject.
    """
    
    roi_fn = os.path.join(resource_dir,'volume_masks','cerebellum_retmaps.nii')
    maskimg = nb.load(roi_fn)
    roimask = (maskimg.get_data()>0) # valid voxels  

    return roimask

def mask_voxels(data,mask_type,sj):
    """
    This function returns a mask bool where 
    1 = valid voxel
    0 = invalid voxel
    """

    if mask_type == 'r2':
        
        # only one mask required
        mask = create_r2_mask(sj,data)
        
    elif mask_type == 'r2_spill':
        
        # create individual masks
        r2mask = create_r2_mask(sj,data)
        spillmask = create_spill_mask()
        
        # combine masks
        mask = r2mask*spillmask
        
    elif mask_type == 'r2_spill_fix':

        # create individual masks
        r2mask = create_r2_mask(sj,data)
        spillmask = create_spill_mask()
        fixmask = create_fix_mask(sj,data)

        # now combine masks
        mask = r2mask*fixmask*spillmask
                
    elif mask_type == 'r2_roi':
        
        # create individual masks
        r2mask = create_r2_mask(sj,data)
        roimask = create_roi_mask()
        
        # combining masks
        mask = r2mask*roimask
        
    return mask


### create a function to determine a subject ranking:

In [190]:
def determine_best_subjects():
    
    """
    This function determines the best subjects based on the median
    r-squared within the retinotopic clusters defined in the average subject.
    """

    roimask = create_roi_mask()

    all_r2 = []
    for sj in range(181):      

        # load the prf results nifti
        fn = os.path.join(data_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii.gz'%sj)
        img = nb.load(fn)
        data = img.get_data()

        # mask r2s with roi mask:
        r2s = np.ravel(data[roimask,dims['r2']])    
        # and get median within these voxels
        all_r2.append(np.nanmedian(r2s))

    # sort subjects based on median r2
    best_subjects = np.argsort(all_r2)[::-1]

    return best_subjects

### setup function to apply different maskings

For the HCP average subject, we'll create the following maskings:

1. r2 mask: r_squared for the avg subjects at 9.8 (determined in HCP retinotopy manuscript)
2. r2 mask + 'spill' mask: add the 'spillover mask' that removes voxels at the border between ventral cortex and the cerebellum, representing smoothed in activity from visual cortex.
3. r2 mask + 'spill' mask + 'fixation' mask: add a mask that excludes pRFs < 0.15 ecc and < 0.15 size. This throws out voxels with pRFs that overlap with the fixation point

Based on the latter mask, separate retinotopic clusters were defined in fslview. For the individual subjects, we will mask the data using the individual subject r2 threshold (2.2), and mask all data that fall outside of the retinotopic clusters as identified in the average subject.

In [194]:
def create_masked_niftis(mask_types,sjs):
    """
    this function actually applies the mask to the data 
    and saves:
    (1) a nifti with all dimensions and 
    (2) a separate nifti for each dimension. This is needed for the matlab SUIT toolbox.
    """
    
    best_subjects = determine_best_subjects()

    for sj in sjs:
        for mask_type in mask_types:

            if sj == 183:
                mask_types =  ['r2','r2_spill','r2_spill_fix']
                rank = 'avg'   
            else:
                mask_types = ['r2_roi']
                # determine subject rank
                rank = str(np.where(best_subjects==sj)[0][0])

            print('now creating masks for subject %d, rank %s, mask %s'%(sj,rank,mask_type))

            # try to create mask dir
            mask_dir = os.path.join(data_dir,'masked_niftis',mask_type)
            if not os.path.isdir(mask_dir): os.mkdir(mask_dir)

            # load the prf results nifti
            fn = os.path.join(data_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii.gz'%sj)        
            img = nb.load(fn)
            data = img.get_data()

            # determine the mask
            mask = mask_voxels(data,mask_type,sj)

            #mask data
            data[~mask] = np.nan

            # save data
            out_fn = os.path.join(mask_dir,'prfresults_subject_rank_%s.nii.gz'%rank)       
            new_data = nb.Nifti1Image(data,affine=img.affine,header=img.header)
            nb.save(new_data,out_fn)

            # save dimensions separately (for surface plots)
            # Note: this has to be an unzipped nifti for SUIT to be able to read it
            for m in ['ang','ecc','rfsize','r2']:
                these_data = data[:,:,:,dims[m]]
                if m == 'ang': # convert to radians, so later conversion is easier
                    these_data = np.radians(these_data)
                out_fn = os.path.join(mask_dir,'prfresults_subject_rank_%s_%s.nii'%(rank,m))        
                new_data = nb.Nifti1Image(these_data,affine=img.affine,header=img.header)
                nb.save(new_data,out_fn)    

#### create masks for average subject

In [192]:
mask_types =  ['r2','r2_spill','r2_spill_fix']
sjs = [183]
create_masked_niftis(mask_types,sjs)

now creating masks for average subject


#### once we defined the retinotopic clusters in the average subjects, we can use this to mask individual subjects files

In [196]:
mask_types =  ['r2_roi']
sjs = range(181)
create_masked_niftis(mask_types,sjs)

now creating masks for subject rank 177, mask r2_roi
now creating masks for subject rank 172, mask r2_roi
now creating masks for subject rank 103, mask r2_roi
now creating masks for subject rank 8, mask r2_roi
now creating masks for subject rank 84, mask r2_roi
now creating masks for subject rank 57, mask r2_roi
now creating masks for subject rank 145, mask r2_roi
now creating masks for subject rank 0, mask r2_roi
now creating masks for subject rank 34, mask r2_roi
now creating masks for subject rank 60, mask r2_roi
now creating masks for subject rank 50, mask r2_roi
now creating masks for subject rank 156, mask r2_roi
now creating masks for subject rank 45, mask r2_roi
now creating masks for subject rank 124, mask r2_roi
now creating masks for subject rank 69, mask r2_roi
now creating masks for subject rank 139, mask r2_roi
now creating masks for subject rank 83, mask r2_roi
now creating masks for subject rank 93, mask r2_roi
now creating masks for subject rank 118, mask r2_roi
now cr